# Algorithmes

In [102]:
import numpy as np
import random 
import simpy
import time
import csv
from collections import namedtuple
from datetime import datetime
import pandas as pd
from statistics import mean
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')

In [103]:
data = pd.read_csv('06-07-2020_04-08-2020.csv', index_col="timestamp_string", parse_dates=True)
data.index.names = ['Date']
data.head()

,identifiant,timestamp,nombre_impressions,prix_impression,prix_total
Date,,,,,
2020-07-06 06:00:00,1,1.594008e+09,11,1,11
2020-07-06 06:00:03,2,1.594008e+09,2,1,2
2020-07-06 06:00:04,3,1.594008e+09,5,1,5
2020-07-06 06:00:05,4,1.594008e+09,1,1,1
2020-07-06 06:00:06,5,1.594008e+09,6,1,6


In [104]:
class Algo:
    def __init__(self, budget_quotidien, nb_heures_jour):
        """Constructeur de la classe"""
        # attributs fixes
        self.budget_quotidien = budget_quotidien
        self.cible = budget_quotidien/(nb_heures_jour*3600)
        # attributs qui evoluent
        self.budget_restant = budget_quotidien
        self.jour = 1
        
    def decision_achat(self, ts, prix):
        """Fonction qui à partir d'un prix et d'un TS retourne une décison d'achat"""
        # TS de la br
        day = datetime.fromtimestamp(ts).day
        # On detecte le changement de jour (reinitialisation du budget restant)
        if self.jour != day:
            self.budget_restant = self.budget_quotidien
        self.jour = day
        month = datetime.fromtimestamp(ts).month
        year = datetime.fromtimestamp(ts).year
        end_day = datetime(year,month,day,19,40,0,0)
        temps_restant =  datetime.timestamp(end_day) - ts
        # calcul de bt
        bt = self.budget_restant / temps_restant
        if (bt >= self.cible or bt<=0) and (self.budget_restant - prix) >= 0:
            achat = True
            self.budget_restant -= prix
        else:
            achat = False
            
        return (achat, bt)         

In [105]:
pacing = Algo(budget_quotidien=3000, nb_heures_jour=14)

In [106]:
achats = list()
budgets = list()
bt = list()
cible = list()
for Date, row in data.iterrows():
    achats.append(pacing.decision_achat(row['timestamp'], row['prix_total'])[0])
    budgets.append(pacing.budget_restant)
    bt.append(pacing.decision_achat(row['timestamp'], row['prix_total'])[1])
    cible.append(pacing.cible)
data['achat'] = achats
data['bt'] = bt
data['cible'] = cible
data['budget'] = budgets

In [107]:
data.groupby(data.index.day).head(1)

,identifiant,timestamp,nombre_impressions,prix_impression,prix_total,achat,bt,cible,budget
Date,,,,,,,,,
2020-07-06 06:00:00,1,1.594008e+09,11,1,11,True,0.060752,0.059524,2989.0
2020-07-07 06:00:00,1419,1.594094e+09,0,1,0,True,0.060976,0.059524,3000.0
2020-07-08 06:00:00,2707,1.594181e+09,4,1,4,True,0.060894,0.059524,2996.0
2020-07-09 06:00:00,4459,1.594267e+09,5,1,5,True,0.060874,0.059524,2995.0
2020-07-10 06:00:00,5865,1.594354e+09,4,1,4,True,0.060894,0.059524,2996.0
2020-07-11 06:00:00,7549,1.594440e+09,5,1,5,True,0.060874,0.059524,2995.0
2020-07-12 06:00:00,9344,1.594526e+09,3,1,3,True,0.060915,0.059524,2997.0
2020-07-13 06:00:00,10589,1.594613e+09,1,1,1,True,0.060955,0.059524,2999.0
2020-07-14 06:00:00,12391,1.594699e+09,0,1,0,True,0.060976,0.059524,3000.0
